In [146]:
"""
Tokenization and sentiment analysis
"""
# conn.cursor().execute("""DROP TABLE linkusersentimenttosubreddit""")
sql_create_linkusersentimenttosubreddit_table = """ CREATE TABLE IF NOT EXISTS linkusersentimenttosubreddit (
                                        userid text,
                                        subredditid text,
                                        rating integer
                                    ); """
conn.cursor().execute(sql_create_linkusersentimenttosubreddit_table)

"""
The snippet below is ran onced - to insert data into a new table
"""

# sid = SentimentIntensityAnalyzer()
# for comment in reddit_comments:
#     array_of_sentences = nltk.sent_tokenize(comment[2])
#     comment_sentiment = 0
#     for sentence in array_of_sentences:
#         ss = sid.polarity_scores(sentence)
#         array_of_tokenized_values = []
#         comment_sentiment += ss['pos']  
#     conn.cursor().execute("INSERT INTO linkusersentimenttosubreddit VALUES (?, ?, ?)", (comment[1], comment[0], comment_sentiment))
#     conn.commit()
newtable = conn.cursor().execute("SELECT * from linkusersentimenttosubreddit").fetchall()
print(newtable)
comments_df = pd.DataFrame(newtable, columns = ['UserID', 'SubredditID', 'Rating'])
R_df = comments_df.pivot(index = 'UserID', columns ='SubredditID', values = 'Rating').fillna(0)
R_df.head()

# # """
# # convert to a numpy array
# # """
R = R_df.as_matrix()
# ratings_mean = np.mean(R)
# R_demeaned = R - ratings_mean.reshape(-1, 1)
# print(R_demeaned)


[('t1_cqug90g', 't5_378oi', 0), ('t1_cqug90h', 't5_2qo4s', 0.423), ('t1_cqug90i', 't5_2cneq', 0), ('t1_cqug90j', 't5_2qh1i', 0), ('t1_cqug90k', 't5_2qh1i', 0)]
[[ 0.       0.       0.       0.     ]
 [-0.10575 -0.10575  0.31725 -0.10575]
 [ 0.       0.       0.       0.     ]
 [ 0.       0.       0.       0.     ]
 [ 0.       0.       0.       0.     ]]


In [147]:
import pandas as pd
import sqlite3
from sqlite3 import Error
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import matplotlib.pyplot as plt
import numpy as np
from scipy.sparse.linalg import svds


def create_connection(db_file):
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Error as e:
        print(e)
 
    return None

conn = create_connection("database.sqlite")
    
reddit_comments = conn.cursor().execute("SELECT subreddit_id, name, body  from May2015 LIMIT 5").fetchall()
reddit_comments[0]

('t5_378oi', 't1_cqug90g', 'くそ\n読みたいが買ったら負けな気がする\n図書館に出ねーかな')

In [1]:
"""
Alright now to perform svd
"""



'\nAlright now to perform svd\n'